In [2]:
import numpy as np 
import os
import cv2
import random
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Function to load and preprocess image data with resized images
def load_and_preprocess_data(directory, categories, img_size):
    data = []
    for category in categories:
        path = os.path.join(directory, category)
        class_num = categories.index(category)
        for file in os.listdir(path):
            filepath = os.path.join(path, file)
            img = cv2.imread(filepath, 0)
            img = cv2.resize(img, (img_size, img_size))  # Resize image
            data.append([img, class_num])
    random.shuffle(data)
    X, y = [], []
    for feature, label in data:
        X.append(feature)
        y.append(label)
    X = np.array(X).reshape(-1, img_size, img_size, 1)
    X = X / 255.0
    y = np.array(y)
    return X, y

# Define directory containing the image dataset
directory = r'C:\Users\ACER\OneDrive\Desktop\cancer\archive\The IQ-OTHNCCD lung cancer dataset\The IQ-OTHNCCD lung cancer dataset'

# Define categories
categories = ['Bengin cases', 'Malignant cases', 'Normal cases']

# Load and preprocess the data with resized images (e.g., resize images to 64x64)
img_size = 64
X, y = load_and_preprocess_data(directory, categories, img_size)

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=10, stratify=y)

# Apply SMOTE to address class imbalance
smote = SMOTE()
X_train_sampled, y_train_sampled = smote.fit_resample(X_train.reshape(-1, img_size*img_size*1), y_train)

# Reshape the data
X_train = X_train.reshape(X_train.shape[0], img_size, img_size, 1)
X_valid = X_valid.reshape(X_valid.shape[0], img_size, img_size, 1)
X_train_sampled = X_train_sampled.reshape(X_train_sampled.shape[0], img_size, img_size, 1)

# Define the model architecture
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=X_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_sampled, y_train_sampled, batch_size=8, epochs=10, validation_data=(X_valid, y_valid))

# Evaluate the model
_, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
_, test_accuracy = model.evaluate(X_valid, y_valid, verbose=0)
print('Train Accuracy: %.2f%%, Test Accuracy: %.2f%%' % (train_accuracy*100, test_accuracy*100))

# Save the trained model to a file
model.save("trained_model.h5")


c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.4030 - loss: 1.0734 - val_accuracy: 0.5164 - val_loss: 0.8688
Epoch 2/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.5599 - loss: 0.9139 - val_accuracy: 0.6291 - val_loss: 0.6631
Epoch 3/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.7326 - loss: 0.7150 - val_accuracy: 0.7491 - val_loss: 0.5441
Epoch 4/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.8351 - loss: 0.5060 - val_accuracy: 0.8800 - val_loss: 0.3496
Epoch 5/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9644 - loss: 0.1240 - val_accuracy: 0.9418 - val_loss: 0.1564
Epoch 6/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.9907 - loss: 0.0357 - val_accuracy: 0.9673 - val_loss: 0.1124
Epoch 7/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.9881 - loss: 0.0343 - val_accuracy: 0.9855 - val_loss: 0.0604
Epoch 8/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.9976 - loss: 0.0150 - val_accu

Train Accuracy: 99.64%, Test Accuracy: 98.18%
